# Summarise building density information

Access NISMOD-DB to download building data and summarise regional floor area and footprint density.

In [ ]:
import configparser
import glob
import json
import os

import pandas
import geopandas
import requests
import shapely.wkt

In [ ]:
def get_auth():    
    # Read connection details
    if 'NISMOD_API_USER' in os.environ and 'NISMOD_API_PASSWORD' in os.environ:
        username = os.environ['NISMOD_API_USER']
        password = os.environ['NISMOD_API_PASSWORD']
    else:
        parser = configparser.ConfigParser()
        parser.read('dbconfig.ini')
        username = parser['nismod-api']['user']
        password = parser['nismod-api']['password']

    return (username, password)

In [ ]:
LAD_CODE = 'E06000042'
BUILDINGS_YEAR = 2017
CACHE_PATH = os.path.join('.', 'db-data')
AUTH = get_auth()

## Get Buildings

In [ ]:
def get_buildings(auth, lad_code, year, force=False):
    try:
        os.mkdir(os.path.join(CACHE_PATH))
    except FileExistsError:
        pass
    buildings_file = os.path.join(CACHE_PATH, "buildings_{}.json".format(lad_code))

    if not os.path.exists(buildings_file) or force:
        r = requests.get(
            'https://www.nismod.ac.uk/api/data/mastermap/buildings',
            auth=auth,
            params={
                'scale': 'lad',
                'area_codes': lad_code,
                'building_year': year
            },
            stream=True
        )
        with open(buildings_file, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

In [ ]:
arc_lad_codes = [
    "E06000031", "E06000032", "E06000042", "E06000055", "E06000056", "E07000004", "E07000005", 
    "E07000006", "E07000007", "E07000008", "E07000009", "E07000010", "E07000011", "E07000012", 
    "E07000150", "E07000151", "E07000152", "E07000153", "E07000154", "E07000155", "E07000156", 
    "E07000177", "E07000178", "E07000179", "E07000180", "E07000181"]

In [ ]:
for arc_lad_code in arc_lad_codes:
    print("Getting", arc_lad_code)
    get_buildings(AUTH, arc_lad_code, BUILDINGS_YEAR)

In [ ]:
dfs = []

for arc_lad_code in arc_lad_codes:
    with open(os.path.join(CACHE_PATH, "buildings_{}.json".format(arc_lad_code))) as fh:
        print("Loading", arc_lad_code)
        lad_buildings = json.load(fh)
        df = geopandas.GeoDataFrame(lad_buildings)
        df.geometry = df.geom.apply(lambda wkt: shapely.wkt.loads(wkt))
        dfs.append(df)

In [ ]:
df = pandas.concat(dfs, axis=0)

In [ ]:
df.floor_area = df.floor_area.astype(float)

In [ ]:
df.drop("geom" ,axis=1, inplace=True)

In [ ]:
df.class_code = df.class_code.apply(lambda d: json.dumps(d))

In [ ]:
df.group_ids = df.group_ids.apply(lambda d: json.dumps(d))

In [ ]:
df.head()

In [ ]:
df.to_file("arc_buildings.gpkg", driver="GPKG")

## Get generic surfaces, associate to buildings

In [ ]:
def get_plots(auth, lad_code, force=False):
    try:
        os.mkdir(os.path.join(CACHE_PATH))
    except FileExistsError:
        pass
    buildings_file = os.path.join(CACHE_PATH, "plots_{}.json".format(lad_code))

    if not os.path.exists(buildings_file) or force:
        r = requests.get(
            'https://www.nismod.ac.uk/api/data/mastermap/areas',
            auth=auth,
            params={
                'scale': 'lad',
                'area_codes': lad_code,
                'classification_codes': 'all'
            },
            stream=True
        )
        with open(buildings_file, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

In [ ]:
for arc_lad_code in arc_lad_codes:
    print("Getting", arc_lad_code)
    get_plots(AUTH, arc_lad_code)

In [ ]:
dfs = []

for arc_lad_code in arc_lad_codes:
    with open(os.path.join(CACHE_PATH, "plots_{}.json".format(arc_lad_code))) as fh:
        df = json.load(fh)
        df = geopandas.GeoDataFrame(df)
        df.geometry = df.geom.apply(lambda wkt: shapely.wkt.loads(wkt))
        df.drop("geom" ,axis=1, inplace=True)
        dfs.append(df)

In [ ]:
lad_plots = pandas.concat(dfs, axis=0)

In [ ]:
lad_plots.descriptive_group = lad_plots.descriptive_group.apply(lambda d: json.dumps(d))

In [ ]:
lad_plots.theme = lad_plots.theme.apply(lambda d: json.dumps(d))

In [ ]:
lad_plots.theme.unique()

In [ ]:
lad_plots.descriptive_group.unique()

In [ ]:
lad_plots = lad_plots[lad_plots.descriptive_group == '["General Surface"]']

In [ ]:
lad_plots.head()

In [ ]:
lad_plots.to_file("arc_surfaces.gpkg", driver="GPKG")

## Summarise buildings

In [ ]:
summary = df[
    ['oa', 'mistral_function_class', 'mistral_building_class', 'floor_area', 'footprint_area', 'res_count']
].groupby(
    ['oa', 'mistral_function_class', 'mistral_building_class']
).sum()
summary.head()

In [ ]:
summary.to_csv('summary_arc_buildings_by_oa.csv')

In [ ]:
summary_lad = df[
    ['lad', 'mistral_function_class', 'mistral_building_class', 'floor_area', 'footprint_area', 'res_count']
].groupby(
    ['lad', 'mistral_function_class', 'mistral_building_class']
).sum()
summary_lad.to_csv('summary_arc_buildings_by_lad.csv')
summary_lad.head()